In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np


c:\Users\rigel\anaconda3\envs\final2\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x # ReLU can be applied before or after adding the input


class ImageRegressionBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        labels = labels.unsqueeze(1).float()  # 레이블을 (batch_size, 1) 형태로 변환하고 Float 타입으로 변경
        out = self(images)                  # Generate predictions
        loss = F.mse_loss(out, labels)      # Calculate loss (Mean Squared Error for regression)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        labels = labels.unsqueeze(1).float()  # 레이블을 (batch_size, 1) 형태로 변환하고 Float 타입으로 변경
        out = self(images)                   # Generate prediction
        loss = F.mse_loss(out, labels)       # Calculate loss
        return {"val_loss": loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        return {"val_loss": epoch_loss} # Combine losses

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))


def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


class ResNet9(ImageRegressionBase):
    def __init__(self, in_channels):
        super().__init__()

        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) # out_dim : 128 x 112 x 112
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv3 = ConvBlock(128, 256, pool=True) # out_dim : 256 x 56 x 56
        self.conv4 = ConvBlock(256, 512, pool=True) # out_dim : 512 x 28 x 28
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # 피처 맵 크기를 1x1로 조정
            nn.Flatten(),
            nn.Linear(512, 1)  # 하나의 실수값을 예측
        )

    def forward(self, xb): # xb is the loaded batch
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out


In [10]:
def load_model(model, path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()  # 평가 모드로 설정
    print(f"Model loaded from {path}")
    return model


In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet 입력 크기에 맞게 조정
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 데이터셋의 평균과 표준편차로 정규화
])

In [14]:
# 모델 로드
model_load_path = r'C:\Users\rigel\Downloads\model.pth'
# model = ResNet9(in_channels=3).cuda() 
model = ResNet9(in_channels=3)
model = load_model(model, model_load_path)

# 이미지를 넣고 예측 수행
def predict_and_compare(model, image_path):
    model.eval()  # 평가 모드 설정
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).cuda()  # 배치 차원 추가 및 GPU로 이동

    with torch.no_grad():
        pred = model(image_tensor)
        pred = pred.cpu().numpy()  # 예측값을 CPU로 이동하여 numpy 배열로 변환

    # 이미지와 예측값 출력
    plt.imshow(image)
    plt.title(f"Predicted: {pred[0][0]:.4f}")
    plt.show()

# 예측 수행
image_path = r'C:\Users\rigel\Downloads\drive-download-20240719T110523Z-001\frame_00031.jpg'  # 이미지 파일 경로
predict_and_compare(model, image_path)


Model loaded from C:\Users\rigel\Downloads\model.pth


AssertionError: Torch not compiled with CUDA enabled

In [15]:
torch.cuda.is_available()

False